# Please unzip previous processe file (audio_wav_16000.zip) where we have converted and downsample mp3 to wav.

# Audio Noise Removal
credit:[Remove Background/Dead Noise] (https://www.kaggle.com/jainarindam/imp-remove-background-dead-noise)

In [1]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import zipfile
import pandas as pd
import numpy as np
import shutil
from scipy.io.wavfile import write
from pydub import AudioSegment
from joblib import Parallel, delayed
import soundfile as sf

import matplotlib.pyplot as plt
from scipy.io import wavfile
import argparse
from glob import glob
from librosa.core import resample, to_mono
from pathlib import Path
import librosa
import librosa.display
from IPython.display import Audio, IFrame, display


import warnings
warnings.filterwarnings('ignore')

# Backgroud noise removal helping functions.

In [2]:
def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/20),min_periods=1,center=True).max()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask, y_mean

def downsample_mono(path, sr):
    rate, wav = wavfile.read(path)
    wav = wav.astype(np.float32, order='F')
    try:
        tmp = wav.shape[1]
        wav = to_mono(wav.T)
    except:
        pass
    wav = resample(wav, rate, sr)
    wav = wav.astype(np.int16)
    return sr, wav

In [3]:
!mkdir -p /tmp/WOLOF_ASR_dataset

# Making temprory directory 

In [4]:
ROOT_PATH = "../input/automatic-speech-recognition-in-wolof/audio_wav_16000/tmp/WOLOF_ASR_dataset/audio_wav_16000"
OUTPUT_DIR = "/tmp/WOLOF_ASR_dataset/noise_remove"

In [5]:
os.mkdir(OUTPUT_DIR)

# Noise removefunction tuned for our data

In [6]:
def noise_remove(filename):
       
    path = f"{ROOT_PATH}/{filename}"
    save_path = f"{OUTPUT_DIR}"
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
    
    if os.path.exists(path):
        try:
            rate, wav = downsample_mono(path,16000)
            mask, env = envelope(wav, rate, threshold=200)
            sf.write(f"{save_path}/{filename}", wav[mask], rate)
        except:
            print(path)

# Checking one sample

In [7]:
rate, wav = downsample_mono("../input/automatic-speech-recognition-in-wolof/audio_wav_16000/tmp/WOLOF_ASR_dataset/audio_wav_16000/009c99482f7d628a32bfcddfa6d04640265344aa6082aedcb55e48052e0a17a4e98f92fe327c6c132c4814551d6501e4f4d90118a9a985bfe6fa603f73e6684f.wav",16000)
mask, env = envelope(wav, rate, threshold=200)
sf.write(f"love.wav",wav[mask],rate)

# Plying noise removed sound

In [8]:
import os
path = "../input/automatic-speech-recognition-in-wolof/audio_wav_16000/tmp/WOLOF_ASR_dataset/audio_wav_16000/"
audio_files = os.listdir(path)

In [9]:
import IPython.display as ipd
import numpy as np
import random

import soundfile as sf
import torchaudio

speech_array, sampling_rate = sf.read("./love.wav")

ipd.Audio(data=speech_array, autoplay=True, rate=16000)

# Running 8 parallel job for multiprocessing of sound files.

In [10]:
Parallel(n_jobs=8, backend="multiprocessing")(
    delayed(noise_remove)(filename) for filename in tqdm(audio_files)
)

  0%|          | 0/10355 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# Ziping file so that it can be uploaded to colab or kaggle. 

In [11]:
!zip -r "./audio_wav_16000.zip" "/tmp/WOLOF_ASR_dataset/noise_remove/"

  adding: tmp/WOLOF_ASR_dataset/noise_remove/ (stored 0%)
  adding: tmp/WOLOF_ASR_dataset/noise_remove/d337355ba2165ec4c59c661c3cf100714934f3b2386126daf77a492af7c8a9c55203ec858f02603c48e18161e8bd1b292a318d9107482024efb2975e9ebe02dd.wav (deflated 12%)
  adding: tmp/WOLOF_ASR_dataset/noise_remove/b9e19e0c9a413daf64cdf75e324c83b059cbca1a35c2d034b38607d79c0b5a6f2433b593dcc8088d4bda58b511d9eec36e56e0c5edc64fb7f2cef149dcb68b9a.wav (deflated 14%)
  adding: tmp/WOLOF_ASR_dataset/noise_remove/f957ef1d4e93948dd4125f610560d3f9b1294dd268b327952a97de47bac0d7dc4a879a8934058963f63850bc0ee783c237e1f1a7327f82b05ce84325d96326a1.wav (deflated 11%)
  adding: tmp/WOLOF_ASR_dataset/noise_remove/9542c9ba2129f93f18ff066284741e263701bd2c8ccc6b6a094a319c9cf70d890a00279fa1c8fb00cf73b09d0d1a0ba2608c11b057734dc3a56a8b3453dc521d.wav (deflated 13%)
  adding: tmp/WOLOF_ASR_dataset/noise_remove/aa18181ef5daef64e0835c53b16d949e4dffe3fe5a1caff0954b50801c68115109d1f4aab1f313d9e57875fc97ddb4e43b710a3b6c4116f3626804061a38b

# Please unzip before runing final notebook, I have also added clean wav files and default Train test files so that reproducibility become easy.

# Next step is to run 3) asr-finetune-wolof-gdrive.ipynb

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=74570277-c535-49fd-8c53-ed6cab466887' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>